# In this notebook, I will merge all Strava and NRC activities in one single DataFrame

In [2]:
import pandas as pd
import numpy as np
import numpy_ext
import gpxpy
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [18]:
directory = 'all_gpx_files'
 
# iterate over files in
# that directory
def gpx_iterate(directory):
    gpx_file_list=[]
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            gpx_file_list.append(f)
    return gpx_file_list

## Great, that's how I'll iterate over every gpx file : )

In [21]:
file_extensions = gpx_iterate(directory)

In [23]:
file_extensions[0:10]

['all_gpx_files/6239779520.gpx',
 'all_gpx_files/1056004315.gpx',
 'all_gpx_files/0b0a39b7-9362-4b1d-9895-6e2c38046bc3.gpx',
 'all_gpx_files/1511363289.gpx',
 'all_gpx_files/1460279536.gpx',
 'all_gpx_files/3468966238.gpx',
 'all_gpx_files/575a84eb-24bc-4241-9651-224c31be8582.gpx',
 'all_gpx_files/6133203641.gpx',
 'all_gpx_files/2950351388.gpx',
 'all_gpx_files/70a4ee20-02f7-4ab7-a400-b425cd759a92.gpx']

In [36]:
df_list=['time','latitude','longitude','elevation','distance','cumulative_distance',\
                           'duration','cumulative_duration','pace_metric']

In [37]:
df_list

['time',
 'latitude',
 'longitude',
 'elevation',
 'distance',
 'cumulative_distance',
 'duration',
 'cumulative_duration',
 'pace_metric']

In [45]:
all_gpx_df = pd.DataFrame(columns={'time','latitude','longitude','elevation','distance','cumulative_distance',\
                           'duration','cumulative_duration','pace_metric'}, index=None)
all_gpx_df

,duration,elevation,cumulative_duration,longitude,cumulative_distance,time,pace_metric,distance,latitude


In [46]:
test=file_extensions[0:10]

In [47]:

for file in test:
    gpx_path = file
    with open(gpx_path) as f:
        gpx = gpxpy.parse(f)
    points = []
    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time': p.time,
                'latitude': p.latitude,
                'longitude': p.longitude,
                'elevation': p.elevation,
            })
    gpx_df = pd.DataFrame.from_records(points)
    # Cumulative distance.
    coords = [(p.latitude, p.longitude) for p in gpx_df.itertuples()]
    gpx_df['distance'] = [0] + [geopy.distance.distance(from_, to).m for from_, to in zip(coords[:-1], coords[1:])]
    gpx_df['cumulative_distance'] = gpx_df.distance.cumsum()
    # Timing.
    gpx_df['duration'] = gpx_df.time.diff().dt.total_seconds().fillna(0)
    gpx_df['cumulative_duration'] = gpx_df.duration.cumsum()
    gpx_df['pace_metric'] = pd.Series((gpx_df.duration / 60) / (gpx_df.distance / 1000)).bfill()
    all_gpx_df.append(gpx_df)
    

In [49]:
gpx_df

,time,latitude,longitude,elevation,distance,cumulative_distance,duration,cumulative_duration,pace_metric
0,2020-05-08 23:27:41.313000+00:00,29.414342,-98.495276,-0.420135,0.000000,0.000000,0.000,0.000,6.208184
1,2020-05-08 23:27:45.236000+00:00,29.414398,-98.495187,191.364375,10.531797,10.531797,3.923,3.923,6.208184
2,2020-05-08 23:27:49.160000+00:00,29.414398,-98.495187,191.163570,0.000000,10.531797,3.924,7.847,inf
3,2020-05-08 23:27:53.083000+00:00,29.414398,-98.495187,191.670894,0.000000,10.531797,3.923,11.770,inf
4,2020-05-08 23:27:57.007000+00:00,29.414524,-98.495035,191.647106,20.305663,30.837460,3.924,15.694,3.220776
...,...,...,...,...,...,...,...,...,...
396,2020-05-08 23:53:35.082000+00:00,29.408768,-98.494740,190.460704,12.690904,4652.081649,3.924,1553.769,5.153297
397,2020-05-08 23:53:39.006000+00:00,29.408675,-98.494764,189.863536,10.536581,4662.618230,3.924,1557.693,6.206947
398,2020-05-08 23:53:42.929000+00:00,29.408566,-98.494813,189.736705,13.063824,4675.682055,3.923,1561.616,5.004915
399,2020-05-08 23:53:46.853000+00:00,29.408475,-98.494864,189.704997,11.237389,4686.919443,3.924,1565.540,5.819857


In [4]:
# Load gpx.
gpx_path = 'strava_gpx_files/6239779520.gpx'
with open(gpx_path) as f:
    gpx = gpxpy.parse(f)

In [5]:
gpx

GPX(tracks=[GPXTrack(name='Lunch Ride', segments=[GPXTrackSegment(points=[...])])])